In [1]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns

import rdkit
from rdkit import Chem, DataStructs
from rdkit.Chem import Draw, rdmolops, AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors


from scipy.stats import pearsonr


# sklearn ML models
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn import svm

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import  LGBMRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.model_selection import KFold

import joblib
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity='all'


/Users/liugrp_m/anaconda3/lib/python3.11/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [2]:
df = pd.read_csv('../01-database-preprocessing-1203dp-to-1115dp/raw/atom_number_wH_sort_1115-backbone-correction-newSMILES.csv')
df.head()

,Nickname,bandgap(eV),c_smiles,newSMILES,Ref.No
0,P3HT,1.93,CCCCCCc1cc(C)sc1C,Cc1cc(C)c(s1)-c1cc(C)c(s1)-c1cc(C)c(s1)-c1cc(C...,S10
1,P3HST,1.82,CCCCCCSc1cc(C)sc1C,CSc1cc(sc1C)-c1sc(cc1SC)-c1sc(cc1SC)-c1sc(cc1S...,S123
2,POPT,1.76,CCCCCCCCc1ccc(-c2cc(C)sc2C)cc1,Cc1cc(c(s1)-c1cc(c(s1)-c1cc(c(s1)-c1cc(c(s1)-c...,S126
3,PT-C1,1.92,CCCCC(CC)COC(=O)c1cc(C)sc1-c1ccc(C)s1,COC(=O)c1cc(C)sc1-c1ccc(s1)-c1cc(C(=O)OC)c(s1)...,S122
4,PT-C2,1.89,CCCCC(CC)COC(=O)c1cc(C)sc1-c1ccc(-c2ccc(C)s2)s1,COC(=O)c1cc(C)sc1-c1ccc(s1)-c1ccc(s1)-c1cc(C(=...,S122


# exp HOMO-LUMO gap

In [3]:
df_exp = df['bandgap(eV)']
df_exp

0       1.93
1       1.82
2       1.76
3       1.92
4       1.89
        ... 
1110    1.68
1111    1.65
1112    1.66
1113    1.52
1114    1.73
Name: bandgap(eV), Length: 1115, dtype: float64

# ECFP descriptors generation

In [4]:
smiles_list = df['c_smiles'].values
len(smiles_list)

1115

In [5]:
# proof and make a list of SMILES
c_smiles = []
for ds in smiles_list:
    try:
        cs = Chem.CanonSmiles(ds)
        c_smiles.append(cs)
    except:
        print('Invalid SMILES:', ds)
len(c_smiles)

1115

In [6]:
nbits=1024

In [7]:
def morgan_fpts(data):
    Morgan_fpts = []
    for i in data:
        mol = Chem.MolFromSmiles(i)
        fpts = AllChem.GetMorganFingerprintAsBitVect(mol,radius=3, nBits=nbits)
        mfpts = np.array(fpts)
        Morgan_fpts.append(mfpts)
    return np.array(Morgan_fpts)

In [8]:
Morgan_fpts = morgan_fpts(c_smiles)

Morgan_fpts.shape


(1115, 1024)

In [9]:
#Name the feature rows as f_0, f_1, f_2...
feature_col=[]
for i in range(Morgan_fpts.shape[1]):
    feature_col.append("f_"+str(i))
    i+=1
Morgan_fingerprints = pd.DataFrame(data = Morgan_fpts, columns=feature_col)
Morgan_fingerprints

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_1014,f_1015,f_1016,f_1017,f_1018,f_1019,f_1020,f_1021,f_1022,f_1023
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,0,1,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1111,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1112,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1113,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
Morgan_fingerprints.to_csv('monomer-1115dp-MorganFP-1024.csv', index=False)

# drop sp3-N polymers + donor-692

In [44]:
# non_alkyl_idx = [46,49,68,79,202,210,217,252,255,262,273,274,318,355,358,375,441,
#                 455,810,812,914,932,934,937,947,1007]

sp3_N_list = [  24,   44,  191,  201,  206,  209,  251,  317,  318,  332,  374,
             381,  388,  454,  913,  931,  936, 1006]
drop_list = sp3_N_list+[691]
print('Total data points: ', 1115-len(drop_list))

Total data points:  1096


In [45]:
Morgan_fingerprints = Morgan_fingerprints[~Morgan_fingerprints.index.isin(drop_list)].reset_index(drop=True)
df_exp = df_exp[~df_exp.index.isin(drop_list)].reset_index(drop=True)

# ML regression

In [46]:
## b. metrics

def acc(y_test,y_pred):
    MSE = mean_squared_error(y_test,y_pred)
    RMSE = MSE ** 0.5
    R2 = r2_score(y_test,y_pred)
#     p = pearsonr(y_test,y_pred.reshape(-1,1)) # y_pred shape = (xxx,)
    r, p_value = pearsonr(y_test,y_pred) # y_pred shape = (xxx,)
    MAE = mean_absolute_error(y_test,y_pred)
    return RMSE, R2, r, MAE

y = df_exp
y

X = Morgan_fingerprints
X

## c. model define

# model = HistGradientBoostingRegressor()
# model = LGBMRegressor(force_col_wise=True, verbose=-1)
# model = GradientBoostingRegressor()
model = XGBRegressor()
# model = AdaBoostRegressor()
# model = RandomForestRegressor()

## d. 10fold-CV plus 10fold-CV average

foldername = 'xgb-10fold-cv-plus-10fold-cv-average'
# foldername = 'hgbr-10fold-cv-plus-10fold-cv-average'
os.makedirs(foldername, exist_ok=True)

xfold=10
kf = KFold(n_splits=xfold, shuffle=True, random_state=42)

scores = []

# save index for train and test of each fold
train_idx_list = []
test_idx_list = []
i=0
for fold_idx, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    train_idx_list.append(train_index)
    test_idx_list.append(test_index)
    
    kf_sub = KFold(n_splits=xfold, shuffle=True, random_state=42)
    for fold_idx_sub, (train_index_sub, test_index_sub) in enumerate(kf_sub.split(X_train)):
        X_kf_train, X_kf_test = X_train.iloc[train_index_sub], X_train.iloc[test_index_sub]
        y_kf_train, y_kf_test = y_train.iloc[train_index_sub], y_train.iloc[test_index_sub]
        

        
        # 训练模型
        model.fit(X_kf_train, y_kf_train)

        # 获取模型预测结果
        y_pred = model.predict(X_kf_test)
        
        # 在验证集上做预测

        RMSE_test, R2_test, r_test, MAE_test = acc(y_kf_test,y_pred)

        scores.append([RMSE_test, R2_test, r_test, MAE_test])
        
        # 保存模型
        model_filename = foldername + f'/model_fold_{fold_idx + 1}_subfold_{fold_idx_sub + 1}.pkl'
        joblib.dump(model, model_filename)
        
        i+=1
        
#         print('model:',i)


scores_df = pd.DataFrame(scores, columns = ['RMSE', 'R2', 'r', 'MAE'])
index_list=[f'fold{i}' for i in range(1,xfold+1)]
index = [index_list[i%xfold] for i in range(xfold**2)]

scores_df.index = index
scores_df.round(3)

## e. load 10-fold cv plus 10-fold cv average

models = []
for fold_idx in range(xfold):
    for fold_idx_sub in range(xfold):
        model_filename = foldername + f'/model_fold_{fold_idx + 1}_subfold_{fold_idx_sub + 1}.pkl'
        model = joblib.load(model_filename)
        models.append(model)
len(models)

scores = []

for i in range(xfold):
    train_index = train_idx_list[i]
    test_index = test_idx_list[i]
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index] 
    
    predictions = []
    
    for j in range(xfold):
        model = models[i*xfold+j]     
               
        y_pred = model.predict(X_test)
        predictions.append(y_pred)
        
#         print('model idx: ',i*xfold+j)
        
    df_predictions = pd.DataFrame(predictions)
    df_predictions = df_predictions.T
    df_predictions['mean'] = df_predictions.iloc[:,:4].mean(axis = 1)
    
    RMSE_test, R2_test, r_test, MAE_test = acc(y_test,df_predictions['mean'])
    scores.append([RMSE_test, R2_test, r_test, MAE_test])


scores_df = pd.DataFrame(scores, columns = ['RMSE', 'R2', 'r', 'MAE'])
scores_df.loc['mean'] = scores_df.iloc[:xfold,:].mean()
scores_df.loc['std'] = scores_df.iloc[:xfold,:].std()
scores_df.round(3)

,RMSE,R2,r,MAE
0,0.092,0.761,0.874,0.064
1,0.119,0.656,0.815,0.086
2,0.126,0.685,0.831,0.085
3,0.127,0.666,0.818,0.076
4,0.106,0.726,0.855,0.072
5,0.115,0.657,0.812,0.078
6,0.106,0.767,0.887,0.079
7,0.113,0.623,0.791,0.070
8,0.108,0.769,0.880,0.077
9,0.100,0.697,0.836,0.064
